In [3]:
file_path = 'hin.txt'

In [15]:
pip install tensorflow

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 272.8/272.8 MB 6.5 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 kB 3.0 MB/s eta 0:00:00
     ---------------------------------------- 4.1/4.1 MB 52.4 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 92.3 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 kB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 1.2/1.2 MB 82.0 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
     --------------------------------------- 24.4/24.4 MB 36.4 MB/s eta 0:00:00
     ------------------------------

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [16]:
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

#tf.enable_eager_execution()

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import time
import string

In [5]:
file_path

'hin.txt'

In [7]:
lines = open(file_path, encoding='UTF-8').read().strip().split('\n')
lines

['Wow!\tवाह!\tCC-BY 2.0 (France) Attribution: tatoeba.org #52027 (Zifre) & #6179147 (fastrizwaan)',
 'Duck!\tझुको!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #6179041 (fastrizwaan)',
 'Duck!\tबतख़!\tCC-BY 2.0 (France) Attribution: tatoeba.org #280158 (CM) & #6179042 (fastrizwaan)',
 'Help!\tबचाओ!\tCC-BY 2.0 (France) Attribution: tatoeba.org #435084 (lukaszpp) & #459377 (minshirui)',
 'Jump.\tउछलो.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179121 (fastrizwaan)',
 'Jump.\tकूदो.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179122 (fastrizwaan)',
 'Jump.\tछलांग.\tCC-BY 2.0 (France) Attribution: tatoeba.org #631038 (Shishir) & #6179123 (fastrizwaan)',
 'Hello!\tनमस्ते।\tCC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #480306 (minshirui)',
 'Hello!\tनमस्कार।\tCC-BY 2.0 (France) Attribution: tatoeba.org #373330 (CK) & #480307 (minshirui)',
 'Cheers!\tवाह-वाह!\tCC-BY 2.0 (France) Attribution: tatoeba.org #487006 (human6

In [19]:
# print("total number of records: ",len(lines))

In [10]:
import string
exclude = set(string.punctuation) # Set of all special characters
remove_digits = str.maketrans('', '', string.digits) # Set of all digits

In [11]:
def preprocess_eng_sentence(sent):
    '''Function to preprocess English sentence'''
    sent = sent.lower() # lower casing
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    sent = sent.translate(remove_digits) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [12]:
def preprocess_hin_sentence(sent):
    '''Function to preprocess Portuguese sentence'''
    sent = re.sub("'", '', sent) # remove the quotation marks if any
    sent = ''.join(ch for ch in sent if ch not in exclude)
    #sent = re.sub("[२३०८१५७९४६]", "", sent) # remove the digits
    sent = sent.strip()
    sent = re.sub(" +", " ", sent) # remove extra spaces
    sent = '<start> ' + sent + ' <end>' # add <start> and <end> tokens
    return sent

In [17]:
# Generate pairs of cleaned English and Portuguese sentences
sent_pairs = []
for line in lines:
    sent_pair = []
    eng = line.rstrip().split('\t')[0]
    hin = line.rstrip().split('\t')[1]
    eng = preprocess_eng_sentence(eng)
    sent_pair.append(eng)
    hin = preprocess_hin_sentence(hin)
    sent_pair.append(hin)
    sent_pairs.append(sent_pair)
sent_pairs

[['<start> wow <end>', '<start> वाह <end>'],
 ['<start> duck <end>', '<start> झुको <end>'],
 ['<start> duck <end>', '<start> बतख़ <end>'],
 ['<start> help <end>', '<start> बचाओ <end>'],
 ['<start> jump <end>', '<start> उछलो <end>'],
 ['<start> jump <end>', '<start> कूदो <end>'],
 ['<start> jump <end>', '<start> छलांग <end>'],
 ['<start> hello <end>', '<start> नमस्ते। <end>'],
 ['<start> hello <end>', '<start> नमस्कार। <end>'],
 ['<start> cheers <end>', '<start> वाहवाह <end>'],
 ['<start> cheers <end>', '<start> चियर्स <end>'],
 ['<start> got it <end>', '<start> समझे कि नहीं <end>'],
 ['<start> im ok <end>', '<start> मैं ठीक हूँ। <end>'],
 ['<start> thanks <end>', '<start> धन्यवाद <end>'],
 ['<start> we won <end>', '<start> हम जीते। <end>'],
 ['<start> awesome <end>', '<start> बहुत बढ़िया <end>'],
 ['<start> come in <end>', '<start> अंदर आ जाओ। <end>'],
 ['<start> get out <end>', '<start> बाहर निकल जाओ <end>'],
 ['<start> go away <end>', '<start> चले जाओ <end>'],
 ['<start> goodbye <end

In [18]:
# This class creates a word -> index mapping (e.g,. "dad" -> 5) and vice-versa 
# (e.g., 5 -> "dad") for each language,
class LanguageIndex():
    def __init__(self, lang):
        self.lang = lang
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()

        self.create_index()

    def create_index(self):
        for phrase in self.lang:
            self.vocab.update(phrase.split(' '))

        self.vocab = sorted(self.vocab)

        self.word2idx['<pad>'] = 0
        for index, word in enumerate(self.vocab):
            self.word2idx[word] = index + 1

        for word, index in self.word2idx.items():
            self.idx2word[index] = word